In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.utils import *
import numpy as np
from sklearn.cluster import KMeans

In [9]:
config = {"dataset": "yelp", "topics": "lda"}
_, model2 = load_models(config)
explainer = shap.Explainer(model2, padding="max_length", truncation=True, max_length=512)

data_train, data_val, data_test = load_data(config)

def tokenize_function(examples):
    output = len(model2.tokenizer.tokenize(examples["sentence"], padding=False, truncation=True, max_length=512)) > 1
    if output == False:
        raise NotImplementedError
    return output

# Filter then select only for blog
if config["dataset"] == "blog":
    indices = list(range(len(data_test)))
    random.seed(316)
    random.shuffle(indices)
    data_test_shap = data_test.select(indices[:1000])
    data_test_shap = data_test_shap.filter(tokenize_function)
else:
    data_test_shap = data_test.filter(tokenize_function)
    indices = list(range(len(data_test_shap)))
    random.seed(316)
    random.shuffle(indices)
    data_test_shap = data_test_shap.select(indices[:1000])

x = [data_test_shap[i]['sentence'] for i in range(len(data_test_shap))]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight'

  0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
print(len(x))

1000


In [58]:
dataset = config["dataset"]

topics, word2idx = get_topics(config, x)
roberta_shap_vals = load(f"shap_vals_distilroberta_{dataset}")
roberta_topic_vals = load(f"topic_vals_distilroberta_{dataset}_lda") #.reshape(-1, 31, 5)
roberta_word_vals = load(f"word_vals_distilroberta_{dataset}_lda")
gpt2_topic_vals = load(f"topic_vals_gpt2_{dataset}_lda")
gpt2_word_vals = load(f"word_vals_gpt2_{dataset}_lda")
print(roberta_topic_vals.shape, gpt2_topic_vals.shape)
assert (roberta_topic_vals.shape == gpt2_topic_vals.shape)

(1000, 31, 5) (1000, 31, 5)


In [59]:
if dataset == "yelp":
    feature = 2
elif dataset == "blog":
    feature = 1
elif dataset == "goemotion":
    feature = 4 # fear

roberta_topic_vals = roberta_topic_vals[:, :, feature]
gpt2_topic_vals = gpt2_topic_vals[:, :, feature]

In [14]:
# Load topics
base_path = "../scripts/"
topics_matrix_df = pd.read_csv(base_path + ("../data/processed_LDA_files/" + dataset + ".csv"))
word2idx = dict(zip(topics_matrix_df["words"], range(len(topics_matrix_df["words"]))))
topics_matrix_df.drop(columns=["words"], inplace=True)
topics_matrix_df = topics_matrix_df.T
topics_raw = topics_matrix_df.to_numpy()
sorted_words = np.argsort(topics_raw, axis=1)
idx2word = {i: word for word, i in word2idx.items()}
for i in range(topics_raw.shape[0]):
    print(i, [idx2word[word_idx] for word_idx in sorted_words[i][::-1][:10]])

0 ['burger', 'fries', 'sandwich', 'cheese', 'burgers', 'sandwiches', 'hot', 'order', 'bread', 'meat']
1 ['told', 'called', 'call', 'said', 'card', 'phone', 'never', 'then', 'day', 'customer']
2 ['bad', 'ever', 'never', 'worst', 'dirty', 'water', 'over', 'down', 'horrible', 'off']
3 ['chicken', 'rice', 'soup', 'thai', 'chinese', 'fried', 'restaurant', 'sauce', 'beef', 'dish']
4 ['bar', 'beer', 'drinks', 'happy', 'drink', 'hour', 'night', 'beers', 'selection', 'bartender']
5 ['room', 'hotel', 'stay', 'rooms', 'vegas', 'casino', 'strip', 'pool', 'stayed', 'night']
6 ['chicken', 'salad', 'sauce', 'bbq', 'meat', 'fried', 'side', 'cheese', 'wings', 'pork']
7 ['vegas', 'las', 'die', 'san', 'und', 'best', 'man', 'der', ':/', 'strip']
8 ["i'm", 'know', 'want', 'its', 'how', 'people', 'say', 'then', 'why', "that's"]
9 ['coffee', 'ice', 'cream', 'chocolate', 'tea', 'cake', 'sweet', 'flavors', 'cup', 'love']
10 ['sushi', 'buffet', 'roll', 'rolls', 'fish', 'crab', 'eat', 'fresh', 'vegas', 'lunch']


In [17]:
# model1, _ = load_models(config)
# explainer = shap.Explainer(model1, padding="max_length", truncation=True, max_length=512)
idx = 5
# tok_sample = explainer.masker.data_transform(x[idx])[0]
# print(tok_sample)
# _, words = word_shap(tok_sample, np.zeros(len(tok_sample)))
# for tok in words:
#     if tok not in word2idx:
#         print(tok)
# print(roberta_word_vals[idx])
# print(roberta_topic_vals[idx])
print(sum(roberta_shap_vals[idx][:, feature]))
print(sum(roberta_topic_vals[idx, :]))
print(x[idx])

-0.019797423992713448
-0.019797423992713444
Poor service (repeated my order three times and it was still wrong), loud music (probably the reason the girl could not hear me and got the order wrong), wrong change (even after I pointed it out, they did not correct it, the cashier walked away), no selection (no pastrami, no swiss, cheddar, only provolone, etc). Why would anyone want to eat here?


In [68]:
X_roberta = np.abs(roberta_topic_vals)
X_gpt2 = np.abs(gpt2_topic_vals)
X_roberta = X_roberta / np.linalg.norm(X_roberta, axis=1, keepdims=True)
X_gpt2 = X_gpt2 / np.linalg.norm(X_gpt2, axis=1, keepdims=True)

# Subtraction of the projection of x_gpt2 onto x_roberta from x_roberta
X = X_roberta - (X_roberta * (X_roberta @ X_gpt2.T).diagonal()[:, np.newaxis])
X = np.abs(X_roberta - X_gpt2)
kmeans = KMeans(n_clusters=100).fit(X)
clabels = kmeans.labels_

diffs = np.zeros(100)
for c in range(100):
    diffs[c] = np.sum(np.linalg.norm(X[clabels == c], axis=1))

most_diff_cluster = np.argsort(diffs)[::-1]
print(diffs)
print(most_diff_cluster[:2])
for most_diff_c in most_diff_cluster[:2]:
    diff = np.sum(np.abs(X[clabels == most_diff_c]), axis=0)
    print(np.argsort(diff)[::-1][:3])
    print(diff[np.argsort(diff)[::-1][:3]])

    print(np.sum(clabels == most_diff_c))
    for idx in np.nonzero(clabels == most_diff_c)[0]:
        print(idx, x[idx])

[ 9.84967122  9.00062675  8.38258876  6.76559253  5.50748469  6.12148405
  1.92599469 13.09285767  5.44900982  2.08237394  2.17327268  3.29173374
  7.66349428  3.54142336  8.01773145  2.46361734  1.98234062  5.25017509
  2.31484687  8.64616615 14.23031169  5.46091624  0.89526168  3.11677419
  2.02413111  3.31866683 10.67838953 23.05621586  9.80238307  3.69973586
  3.97712908  1.30752074  2.80857745  1.86428453  2.03679081  4.35921041
  6.00898107  3.57985698  1.02705776  2.86590427  4.91210837 11.08451294
  5.0710424   4.5113924   1.32240558  4.53621562 11.92008198  7.25864522
  3.14001228 13.2654927   0.94529108  1.05590874  1.22908021 15.90099752
  1.28210165  4.0681868   2.37367468  0.78134398  1.17736915  2.52792474
  5.44134047  1.80068526  1.28522205  1.59817144  8.78757923  1.61656587
  3.89210717  2.77839389  1.15508299 14.26044177  0.77140474  1.04376697
  1.00050502 10.79036622  1.32374182  2.23599525  1.1843321   0.9351678
  1.28427155  4.96311352  1.08679982  1.11090085  3.

In [46]:
diffs = np.sum(np.abs(roberta_topic_vals - gpt2_topic_vals), axis=1)
biggest_diff = np.argsort(diffs)
print(diffs[biggest_diff[-5:]])

[0.91939705 0.91943695 0.94024349 0.9600132  0.98465604]


In [56]:
ind = 29 #biggest_diff[-1]
print(x[ind])
topic_names = np.array([f"topic_{i}" for i in range(36)])
topic_values, sorted_names = sort_shap(roberta_topic_vals[ind], topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")
print(np.sum(topic_values))

print()
topic_values, sorted_names = sort_shap(gpt2_topic_vals[ind], topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")
print(np.sum(topic_values))

Very disappointing. \n\nCame early Saturday evening, around 5:30, and it was packed.  Wait for a table for 2 was 45 mins. You can't make reservations despite what their (infuriating and confusing) automated answering machine said.\n\nOk, no problem.  I'm willing to wait for good food and to try a new place.  Luckily, we snagged two seats at the bar within 10 mins.  It was all downhill from here...\n\nCocktails were awful.  They don't carry Tito's, so I went with a Stoli (up with a twist).  Not good.  The perfect vodka martini should be ice cold with shards of ice crystals and citrus oils.  Mine was cold-ish... and had a thick awkward corkscrew of rind balanced on the rim.  Whatever.  We moved on to the specialty \"cockytails.\"  DIABETES. seriously. we tried the blueberry skinny tea and cool as a cucumber and they both tasted like sugar water.  The one/only good thing was the jalapeno bacon moonshine.  We took shots of that and I would recommend.  Very jalapeno-y and bacon-y and... yah

In [60]:
topic_names = np.array([f"topic_{i}" for i in range(36)])
roberta_feat_imp = np.sum(np.abs(roberta_topic_vals), axis=0)
gpt2_feat_imp = np.sum(np.abs(gpt2_topic_vals), axis=0)

topic_values, sorted_names = sort_shap(roberta_feat_imp, topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
# for i in range(5, 0, -1):
#     print(f"{sorted_names[-i]}: {topic_values[-i]}")

print()
topic_values, sorted_names = sort_shap(gpt2_feat_imp, topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
# for i in range(5, 0, -1):
#     print(f"{sorted_names[-i]}: {topic_values[-i]}")

topic_13: 1.345522683634491
topic_7: 2.0354761906008134
topic_21: 3.0041095573802608
topic_28: 3.0228035622768745
topic_14: 3.046536669753616

topic_13: 1.3802236235986711
topic_7: 2.616008650112999
topic_18: 4.027793597408098
topic_29: 4.144087150235778
topic_27: 4.201744314700697
